# Flask-SQLAlchemy ORM - Praca z danymi (komendy DML i DQL)

### Co potrzebujemy do uruchomienia tego notebooka?
* jupyter
* Flask-SQLAlchemy

<code>$ pip install jupyter Flask-SQLAlchemy</code>

Więcej informacji znajdziesz [tutaj](https://stackoverflow.com/questions/39773125/use-flask-sqlalchemy-models-in-jupyter-notebook)

In [ ]:
### Połączenie z bazą 

In [1]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.orm import DeclarativeBase

class Base(DeclarativeBase):
  pass

db = SQLAlchemy(model_class=Base)

In [2]:
# create the app
app = Flask(__name__)

# configure the SQLite database, relative to the app instance folder
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///database.sqlite3"

# initialize the app with the extension (instance folder is created)
db.init_app(app)

## Definicja modelu

<code>class Task(models.Model):
    name = models.CharField(max_length=64)
</code>
<code>
    def __str__(self):
        return f"{self.name}"
</code>

In [3]:
from sqlalchemy import Integer, String
from sqlalchemy.orm import Mapped, mapped_column

class Task(db.Model):
    id: Mapped[int] = mapped_column(Integer, primary_key=True)
    name: Mapped[str] = mapped_column(String)

In [4]:
app.app_context().push()

In [5]:
db.create_all()

# optionally
# with app.app_context():
    # db.create_all()

## C - CREATE (CRUD) - DML (Data Manipulation Language)

### Klauzula INSERT

In [9]:
# Metoda I - metoda add sesji

task = Task(name="Sprzątanie")
db.session.add(task)
db.session.commit()

In [10]:
# Metoda II - metoda add_all sesji

db.session.add_all([
    Task(name="Pisanie"),
    Task(name="Zamiatanie"),
    Task(name="Malowanie")
])
db.session.commit()

In [11]:
# Przed przejściem do litery R dodajmy jeszcze kilka wpisów do tabeli, żeby mieć 
# co analizować podczas poznawania instrukcji DQL.

db.session.add_all([
    Task(name="Szukanie"),
    Task(name="Szukanie"),
    Task(name="Programowanie"),
    Task(name="Pływanie"),
    Task(name="Pranie"),
    Task(name="Dodawanie"),
])
db.session.commit()

## R - Read (CRUD) - DQL (Data Query Language)

### Klauzula SELECT

Operacja READ w SQL to instrukcja SELECT z całą swoją rozbudowaną składnią i operatorami takimi jak: LIKE, GROUP_BY, ORDER_BY, HAVING, IN, JOIN, UNION, ...
Metoda **.query** sesji posiada odpowiednie metody implementujące część tych instrukcji.

In [12]:
print(dir(Task))

['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__fsa__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__mapper__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__table__', '__table_cls__', '__tablename__', '__weakref__', '_is_protocol', '_sa_class_manager', '_sa_registry', 'id', 'metadata', 'name', 'query', 'query_class', 'registry']


In [13]:
# metoda query sesji
print(db.session.query(Task))

SELECT task.id AS task_id, task.name AS task_name 
FROM task


In [14]:
print(type(db.session.query(Task)))

<class 'flask_sqlalchemy.query.Query'>


In [15]:
print(dir(db.session.query(Task)))

['__annotations__', '__class__', '__class_getitem__', '__clause_element__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_add_context_option', '_all_selected_columns', '_annotate', '_annotations', '_annotations_cache_key', '_assert_no_memoizations', '_auto_correlate', '_clone', '_clone_annotations_traverse_internals', '_compile_context', '_compile_options', '_compile_state', '_copy_internals', '_correlate', '_current_path', '_deannotate', '_distinct', '_distinct_on', '_effective_plugin_target', '_enable_assertions', '_entity_from_pre_ent_zero', '_executable_traverse_internals', '_execution_options', '_filter_by_ze

In [16]:
# metoda `all` obiektu `query`
result = db.session.query(Task).all()
print(result)

[<Task 1>, <Task 2>, <Task 3>, <Task 4>, <Task 5>, <Task 6>, <Task 7>, <Task 8>, <Task 9>, <Task 10>]


In [17]:
print(type(result))

<class 'list'>


In [18]:
print(result[0])

<Task 1>


In [19]:
print(type(result[0]))

<class '__main__.Task'>


In [20]:
for entry in result:
    print(entry)

<Task 1>
<Task 2>
<Task 3>
<Task 4>
<Task 5>
<Task 6>
<Task 7>
<Task 8>
<Task 9>
<Task 10>


In [21]:
for entry in result:
    print(entry.name)

Sprzątanie
Pisanie
Zamiatanie
Malowanie
Szukanie
Szukanie
Programowanie
Pływanie
Pranie
Dodawanie


In [22]:
# jeżeli nie chcemy z bazy wyciągać wszystkiego, tylko wartości w wybranych kolumnach, to kolumny podajemy jako parametry metody query.
result = db.session.query(Task.name)
print(result)

SELECT task.name AS task_name 
FROM task


In [23]:
print(result[0])

('Sprzątanie',)


In [24]:
print(type(result[0]))

<class 'sqlalchemy.engine.row.Row'>


In [25]:
print(dir(result[0]))

['__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__pyx_vtable__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__setattr__', '__setstate__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '_abc_impl', '_asdict', '_data', '_fields', '_filter_on_values', '_get_by_key_impl_mapping', '_is_protocol', '_key_to_index', '_mapping', '_op', '_parent', '_special_name_accessor', '_t', '_to_tuple_instance', '_tuple', '_values_impl', 'count', 'index', 't', 'tuple']


In [26]:
print(result[0][0])

Sprzątanie


In [27]:
print(result[0].name)

Sprzątanie


Możemy też użyć metody query modelu.

In [28]:
print(Task.query)

SELECT task.id AS task_id, task.name AS task_name 
FROM task


In [29]:
print(type(Task.query))

<class 'flask_sqlalchemy.query.Query'>


In [30]:
tasks = Task.query.all()

print(tasks)

[<Task 1>, <Task 2>, <Task 3>, <Task 4>, <Task 5>, <Task 6>, <Task 7>, <Task 8>, <Task 9>, <Task 10>]


Metoda all zwraca listę instancji modelu.

Wśród wielu metod udostępnianych przez query można znaleźć all, filter, filter_by, exists, union, get, firstFD, order_by, ...

Popatrzmy na wybrane

In [32]:
tasks = Task.query.order_by('name')

print(tasks)

SELECT task.id AS task_id, task.name AS task_name 
FROM task ORDER BY task.name


In [33]:
print(type(tasks))

<class 'flask_sqlalchemy.query.Query'>


Metoda order_by też zwraca obiekt klasy Query na którym możemy dalej działać. Skoro to co zwraca metoda order_by to obiekt klasy Query, to ten obiekt posiada takie metody jak filter_by, order_by, ... Wynika z tego, że te metody możemy łańcuchować, tzn. wywoływać jedna po drugiej.

In [34]:
tasks = Task.query.order_by('name').filter_by(name='Szukanie')

print(tasks)

SELECT task.id AS task_id, task.name AS task_name 
FROM task 
WHERE task.name = ? ORDER BY task.name


In [35]:
print(type(tasks))

<class 'flask_sqlalchemy.query.Query'>


I znów Query. Struktura umożliwiająca łańcuchowanie metod na tyle często pojawia się w programowaniu, że posiada nawet swoją nazwę. Mówimy, że Query implementuje wzorzec fluent interface (płynny interfejs).

Ale nie wszystkie metody Query zwracają Query. Na przykład metody all, first i one zwracają odpowiednio listę wszystkich, pierwszy i jeden element (czyli instancje modelu). Takie metody nie zwracają obiektu klasy Query (nie implementują wzorca fluent interface) i dlatego po ich użyciu nie można już użyć żadnej innej metody obiektu Query do łańcuchowania.

In [36]:
tasks = Task.query.order_by('name').limit(7).all()
print(tasks)

[<Task 10>, <Task 4>, <Task 2>, <Task 9>, <Task 7>, <Task 8>, <Task 1>]


In [37]:
task = Task.query.order_by('name').limit(7).first()
print(task)

<Task 10>


In [ ]:
task = Task.query.order_by(Task.name.desc()).limit(7).first()
print(task)

Metody klasy Query implementują wzorzec lazy evaluation. Są wykonywane dopiero w momencie konsumowania. Konsumowanie polega na użyciu wartości zwracanych w zapytaniu i jest równoznaczne z wykonaniem zapytania na bazie. Skonsumować obiekt klasy Query można np. poprzez wywołanie jednej z konsumujących metod (all, first, ...), przeiterowanie się po nim czy np. zrzutowaniu obiektu na listę. Jest to moment, w którym wyniki są potrzebne (do wyświetlenia lub np. zapisania do pamięci) i nie można dłużej zwlekać z wykonaniem zapytania na bazie.

In [38]:
for item in tasks:
    print(item.name)

Dodawanie
Malowanie
Pisanie
Pranie
Programowanie
Pływanie
Sprzątanie


### Dostęp do wartości w poszczególnych kolumnach wpisu

Do wartości w poszczególnych kolumnach wpisu dostajemy się poprzez notacją obiektową (odwołujemy się do atrybutu instancji modelu). Jaką wartość w kolumnie name ma ostatni wpis z tabelki Task?

In [39]:
print(task.name)

Dodawanie


### Filtry - metody filter, filter_by (klauzula WHERE)

#### Metoda I - filter_by

Metoda ```filter_by``` służy do wykonywania prostych zapytań.

In [40]:
query = Task.query.filter_by(name="Szukanie")
print(query)
print(type(query))
result = query.all()
print(result)

SELECT task.id AS task_id, task.name AS task_name 
FROM task 
WHERE task.name = ?
<class 'flask_sqlalchemy.query.Query'>
[<Task 5>, <Task 6>]


Parametrem funkcji `filter_by` jest kolumna, po której filtrujemy. Możemy filtrować po kilku kolumnach jednocześnie.

In [41]:
query = Task.query.filter_by(id=16, name="Szukanie")
print(query)
result = query.all()
print(result)

SELECT task.id AS task_id, task.name AS task_name 
FROM task 
WHERE task.id = ? AND task.name = ?
[]


filter_by zwraca listę instancji modelu spełniających kryterium wyszukiwania. Może być pusta.

In [42]:
query = Task.query.filter_by(name="Coś czego nie ma w bazie")
print(query)
result = query.all()
print(result)

SELECT task.id AS task_id, task.name AS task_name 
FROM task 
WHERE task.name = ?
[]


Może być jednoelementowa

In [43]:
query = Task.query.filter_by(name="Pranie")
print(query)
result = query.all()
print(result)

SELECT task.id AS task_id, task.name AS task_name 
FROM task 
WHERE task.name = ?
[<Task 9>]


#### Metoda II - filter

Metoda ```filter``` Query wykonuje to samo co metoda filter_by (odpowiada klauzuli WHERE). Posiada inne api.

In [44]:
query = Task.query.filter(Task.name=="Szukanie")
print(query)

SELECT task.id AS task_id, task.name AS task_name 
FROM task 
WHERE task.name = ?


Też zwraca obiekt klasy Query.

In [45]:
print(type(query))

<class 'flask_sqlalchemy.query.Query'>


na którym możemy dalej działać.

In [46]:
result = query.all()
print(result)

[<Task 5>, <Task 6>]


ale jako parametr wejściowy przyjmuje całe wyrażenie, dlatego pozwala na formułowaniu bardziej złożonych zapytań. W metodzie filter można używać kilku
operatorów, takich jak:

In [47]:
# eq
query = Task.query.filter(Task.name=="Malowanie")
print(query.statement)
result = query.all()
print(result)

SELECT task.id, task.name 
FROM task 
WHERE task.name = :name_1
[<Task 4>]


In [48]:
# not eq
q = Task.query.filter(Task.name!="Malowanie")
print(q.statement)
result = q.all()
print(result)

SELECT task.id, task.name 
FROM task 
WHERE task.name != :name_1
[<Task 1>, <Task 2>, <Task 3>, <Task 5>, <Task 6>, <Task 7>, <Task 8>, <Task 9>, <Task 10>]


In [49]:
# like
q = Task.query.filter(Task.name.like("%wanie"))
print(q.statement)
result = q.all()
print(result)

SELECT task.id, task.name 
FROM task 
WHERE task.name LIKE :name_1
[<Task 4>, <Task 7>, <Task 8>, <Task 10>]


In [50]:
# in
q = Task.query.filter(Task.name.in_(["Malowanie", "Szukanie", "Pływanie"]))
print(q.statement)
result = q.all()
print(result)

SELECT task.id, task.name 
FROM task 
WHERE task.name IN (__[POSTCOMPILE_name_1])
[<Task 4>, <Task 5>, <Task 6>, <Task 8>]


In [51]:
# not in
q = Task.query.filter(~Task.name.in_(["Malowanie", "Szukanie", "Pływanie"]))
print(q.statement)
result = q.all()
print(result)

SELECT task.id, task.name 
FROM task 
WHERE (task.name NOT IN (__[POSTCOMPILE_name_1]))
[<Task 1>, <Task 2>, <Task 3>, <Task 7>, <Task 9>, <Task 10>]


In [52]:
# gt, lt, gte, lte
q = Task.query.filter(Task.id > 5)
print(q.statement)
result = q.all()
print(result)

q = Task.query.filter(Task.id <= 4)
print(q.statement)
result = q.all()
print(result)

SELECT task.id, task.name 
FROM task 
WHERE task.id > :id_1
[<Task 6>, <Task 7>, <Task 8>, <Task 9>, <Task 10>]
SELECT task.id, task.name 
FROM task 
WHERE task.id <= :id_1
[<Task 1>, <Task 2>, <Task 3>, <Task 4>]


In [53]:
# and v1
q = Task.query.filter(Task.name.like("%wanie"), Task.id >=6)
print(q.statement)
result = q.all()
print(result)

SELECT task.id, task.name 
FROM task 
WHERE task.name LIKE :name_1 AND task.id >= :id_1
[<Task 7>, <Task 8>, <Task 10>]


In [54]:
# and v2
from sqlalchemy import and_

q = Task.query.filter(and_(Task.name.like("%wanie"), Task.id >=6))
print(q.statement)
result = q.all()
print(result)

SELECT task.id, task.name 
FROM task 
WHERE task.name LIKE :name_1 AND task.id >= :id_1
[<Task 7>, <Task 8>, <Task 10>]


In [55]:
# and v3
q = Task.query.filter(
    (Task.name.like("%wanie")) & (Task.id >= 6)
)  # najlepiej warunki zamykać w nawiasach
print(q.statement)
result = q.all()
print(result)

SELECT task.id, task.name 
FROM task 
WHERE task.name LIKE :name_1 AND task.id >= :id_1
[<Task 7>, <Task 8>, <Task 10>]


In [56]:
# or v1
from sqlalchemy import or_

q = Task.query.filter(or_(Task.name.like("%wanie"), Task.id >= 6))
print(q.statement)
result = q.all()
print(result)

SELECT task.id, task.name 
FROM task 
WHERE task.name LIKE :name_1 OR task.id >= :id_1
[<Task 4>, <Task 6>, <Task 7>, <Task 8>, <Task 9>, <Task 10>]


In [57]:
# or v2
q = Task.query.filter(
    (Task.name.like("%wanie")) | (Task.id >= 6)                                 
)
print(q.statement)
result = q.all()
print(result)

SELECT task.id, task.name 
FROM task 
WHERE task.name LIKE :name_1 OR task.id >= :id_1
[<Task 4>, <Task 6>, <Task 7>, <Task 8>, <Task 9>, <Task 10>]


#### Metoda III - get

W odróżnieniu od metody filter, metoda get zwraca instancję jedną instancje modelu. Jako parametr przyjmuje wartość klucza głównego w tabeli.

In [58]:
q = Task.query.get(1)  # get - getting task by primary key
print(task)
print(type(task))

<Task 10>
<class '__main__.Task'>


C:\Users\User\AppData\Local\Temp\ipykernel_3356\3265998587.py:1: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  q = Task.query.get(1)  # get - getting task by primary key


Przedawnione, od wersji 2.0 twórcy zalecają używanie metody get sesji, a nie modelu.

In [59]:
q = db.session.get(Task, 1)
print(q)

<Task 1>


Jeżeli w tabeli nie będzie takiego rekordu metoda get zwraca None.

In [60]:
task = db.session.get(Task, 100)
print(task)

None


Podsumowując, metoda get w odróżnieniu od metody filter:
* zwraca instancję modelu (a nie obiekty klasy Query)
* jeżeli w wyniku filtrowania otrzymamy pustą odpowiedź zwróci None (a nie pustą listę)

### Indeksowanie, wycinki (operatory LIMIT i OFFSET)

Klasa Query wspiera indeksowanie oraz wycinki

Znajdźmy czwarty wpis w tabeli Task (indeksowanie od 0).

In [66]:
task = Task.query.all()[1]
print(task)
print(type(task))  # instancja modelu

<Task 2>
<class '__main__.Task'>


Znajdźmy pięc pierwszych wpisów w tabeli Task.

In [62]:
tasks = Task.query.all()[:5]
print(tasks)

[<Task 1>, <Task 2>, <Task 3>, <Task 4>, <Task 5>]


Znajdźmy wszystkie wpisy w tabeli Task poza pięcioma pierwszymi.

In [63]:
tasks = Task.query.all()[5:]
print(tasks)

[<Task 6>, <Task 7>, <Task 8>, <Task 9>, <Task 10>]


Znajdźmy co drugi wpis z tabeli Task (wycinki)

In [64]:
tasks = Task.query.all()[::2]
print(tasks)
print(type(tasks))

[<Task 1>, <Task 3>, <Task 5>, <Task 7>, <Task 9>]
<class 'list'>
